# Tumor detection model
This model will detect tumors from MRI image

## Configures (dataset, model ,labels)

In [3]:
# dataset
data_set_path = "/kaggle/input/mri-images/Data/*"
image_files_pattern = "*.jpg"
image_size = 256, 256

# model
model_input_size = 256, 256, 3
model_save_path = "/kaggle/working/tumor_detector_model.h5"
learning_rate = 0.001
epochs = 50
batch_size = 32

# labels
label_count = 4

## Import needed libaries

In [4]:
# import data handling tools
import random
import glob
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.applications.efficientnet import EfficientNetB3,preprocess_input
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau
from tensorflow.keras.models import load_model

# Load dataset from data_set_path

In [5]:
images_paths = []
labels = []

def get_image_label_from_name(image_name):
    image_name_parts = image_name.split("_")
    image_label = image_name_parts[0]
    return image_label

def get_image_name_from_path(image_path):
    image_path_parts = image_path.split("/")
    image_name = image_path_parts[-1]
    return image_name

for image_path in glob.glob(f"{data_set_path}/{image_files_pattern}"):
    images_paths.append(image_path)

    image_name = get_image_name_from_path(image_path)
    label = get_image_label_from_name(image_name)
    labels.append(label)

## Create df with [images_paths,labels] colums

In [6]:
df_images_paths = pd.Series(images_paths, name= 'images_paths')
df_labels = pd.Series(labels, name='labels')

df = pd.concat([df_images_paths, df_labels], axis= 1)
df.head()

,images_paths,labels
0,/kaggle/input/mri-images/Data/pituitary_tumor/...,P
1,/kaggle/input/mri-images/Data/pituitary_tumor/...,P
2,/kaggle/input/mri-images/Data/pituitary_tumor/...,P
3,/kaggle/input/mri-images/Data/pituitary_tumor/...,P
4,/kaggle/input/mri-images/Data/pituitary_tumor/...,P


## Split data to train and test

In [7]:
train_df, test_df = train_test_split(df,  train_size= 0.75, shuffle= True, random_state= 123)

## Create image augmantion for train (Random lowering qality)

In [8]:
def random_lower_qality_augmentation(image):
    r = random.random()
    if r >= 0.25:
        return image

    h, w = image.shape[:2]
    image = cv2.resize(image, (w//2, h//2),
                             interpolation=cv2.INTER_LINEAR)
    image = cv2.resize(
        image, (w, h), interpolation=cv2.INTER_LINEAR)

    image = preprocess_input(image)
    return image


train_generator = ImageDataGenerator(
    rotation_range=10,
    brightness_range=(0.85, 1.15),
    width_shift_range=0.002,
    height_shift_range=0.002,
    shear_range=12.5,
    zoom_range=0,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode="nearest",
    preprocessing_function = random_lower_qality_augmentation
)

train_flow = train_generator.flow_from_dataframe( train_df,class_mode="categorical", x_col= 'images_paths', y_col= 'labels', target_size= image_size,color_mode= 'rgb', shuffle= True, batch_size= batch_size)

Found 2322 validated image filenames belonging to 4 classes.


## Create image augmantion for test (only preprocess)

In [ ]:
simple_generator = ImageDataGenerator(
    preprocessing_function = preprocess_input
)

test_flow = simple_generator.flow_from_dataframe( test_df,class_mode="categorical", x_col= 'images_paths', y_col= 'labels', target_size= image_size,color_mode= 'rgb', shuffle= True, batch_size= batch_size)

## Take look at images

In [ ]:
g_dict = train_flow.class_indices    
classes = list(g_dict.keys())      
images, labels = next(train_flow)      

plt.figure(figsize= (20, 10))

for i in range(8):
    plt.subplot(2, 4, i + 1)
    image = images[i] / 255
    plt.imshow(image)
    index = np.argmax(labels[i])
    class_name = classes[index]  
    plt.title(class_name, color= 'black', fontsize= 14)

## Create model, base on pretrain EfficientNetB3 trained on imagenet dataset

In [ ]:
base_model = EfficientNetB3(include_top= False, weights= "imagenet", input_shape= model_input_size, pooling= 'max')

model = Sequential([
    base_model,
    BatchNormalization(),
    Dense(256, activation= 'relu',kernel_regularizer=l1_l2(l1=0.01, l2=0.01)),
    Dropout(0.5),
    Dense(label_count, activation= 'softmax')
])

model.compile(Adam(learning_rate= 0.001), loss= 'categorical_crossentropy', metrics= ['accuracy'])

model.summary()

## Create callbacks for training (EarlyStopping,ReduceLearningRate,BestModelSaver)

In [ ]:
early_stopping = EarlyStopping(patience=50, monitor='val_loss', restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(factor=0.2, patience=3, monitor='val_loss', verbose=1)

model_checkpoint_callback = ModelCheckpoint(
    filepath=model_save_path,  
    monitor='val_accuracy',  
    mode='max', 
    save_best_only=True,  
    verbose=1 
)

## Train model with train data

In [ ]:
history = model.fit(train_flow,
                    epochs = epochs,
                    validation_data = test_flow,
                    steps_per_epoch=train_df.shape[0] // batch_size,
                    validation_steps = test_df.shape[0] // batch_size,
                    callbacks=[early_stopping, reduce_lr,model_checkpoint_callback])

## Plot training steps statistics

In [ ]:
pd.DataFrame(history.history).plot(figsize=(16, 9))

## Load model from model_save_path

In [ ]:
model = load_model(model_save_path)

## Evaluate model with train data with training augmant

In [ ]:
model.evaluate(train_flow)

## Evaluate model with train data with no augmantation

In [ ]:
train_flow = simple_generator.flow_from_dataframe( train_df,class_mode="categorical", x_col= 'images_paths', y_col= 'labels', target_size= image_size,color_mode= 'rgb', shuffle= True, batch_size= batch_size)
model.evaluate(train_flow)

## Evaluate model with test data no augmantation

In [ ]:
model.evaluate(test_flow)

## Create custom augmentation for make imager have less qaulity

In [ ]:
def lower_qality_augmentation(image):
    h, w = image.shape[:2]
    image = cv2.resize(image, (w//2, h//2),
                             interpolation=cv2.INTER_LINEAR)
    image = cv2.resize(
        image, (w, h), interpolation=cv2.INTER_LINEAR)

    image = preprocess_input(image)
    return image


lower_generator = ImageDataGenerator(
    preprocessing_function = lower_qality_augmentation
)

## Evaluate model with train data with lower qaulity

In [ ]:
lower_train_flow = lower_generator.flow_from_dataframe( train_df,class_mode="categorical", x_col= 'images_paths', y_col= 'labels', target_size= image_size,color_mode= 'rgb', shuffle= True, batch_size= batch_size)

In [ ]:
model.evaluate(lower_train_flow)

## Evaluate model with test data with lower qaulity

In [ ]:
lower_test_flow = lower_generator.flow_from_dataframe( test_df,class_mode="categorical", x_col= 'images_paths', y_col= 'labels', target_size= image_size,color_mode= 'rgb', shuffle= True, batch_size= batch_size)

In [ ]:
model.evaluate(lower_test_flow)

## Class indices

In [10]:
train_flow.class_indices

{'G': 0, 'M': 1, 'N': 2, 'P': 3}